In [ ]:
import json
from typing import List, Dict

class Order:
    def __init__(self):
        self.order_items = []

    def to_dict(self):
        return {
            "order_items": [item.to_dict() for item in self.order_items]
        }

class OrderItem:
    def __init__(self, drink: str, size: str, temperature: str, quantity: int, add_ons: Dict[str, int], customer_requests: List[str]):
        self.drink = drink
        self.size = size
        self.temperature = temperature
        self.quantity = quantity
        self.add_ons = add_ons
        self.customer_requests = customer_requests

    def to_dict(self):
        return {
            "drink": self.drink,
            "size": self.size,
            "temperature": self.temperature,
            "quantity": self.quantity,
            "add_ons": self.add_ons,
            "customer_requests": self.customer_requests
        }

class OrderManager:
    def __init__(self):
        self.current_order = None

    def create_new_order(self):
        self.current_order = Order()
        return {"status": "success", "message": "New order created."}

    def add_item(self, item: OrderItem):
        if self.current_order:
            self.current_order.order_items.append(item)
            return {"status": "success", "message": "Item added successfully."}
        else:
            return {"status": "error", "message": "No active order. Please create a new order first."}

    def update_item(self, item_index: int, updated_item: OrderItem):
        if self.current_order:
            if 0 <= item_index < len(self.current_order.order_items):
                self.current_order.order_items[item_index] = updated_item
                return {"status": "success", "message": "Item updated successfully."}
            else:
                return {"status": "error", "message": "Item index out of range."}
        else:
            return {"status": "error", "message": "No active order. Please create a new order first."}

    def cancel_order(self):
        if self.current_order:
            self.current_order = None
            return {"status": "success", "message": "Order cancelled successfully."}
        else:
            return {"status": "error", "message": "No active order to cancel."}

    def remove_item(self, item_index: int):
        if self.current_order:
            if 0 <= item_index < len(self.current_order.order_items):
                removed_item = self.current_order.order_items.pop(item_index)
                return {"status": "success", "message": f"Item '{removed_item.drink}' removed successfully."}
            else:
                return {"status": "error", "message": "Item index out of range."}
        else:
            return {"status": "error", "message": "No active order. Please create a new order first."}

    def get_order_summary(self):
        if self.current_order:
            return {"status": "success", "order_summary": self.current_order.to_dict()}
        else:
            return {"status": "error", "message": "No active order."}

# Example usage of the system

# Initialize OrderManager
order_manager = OrderManager()

# Create a new order
create_response = order_manager.create_new_order()
print(json.dumps(create_response, ensure_ascii=False))

# Add an item to the order
item1 = OrderItem(
    drink="아메리카노",
    size="라지",
    temperature="핫",
    quantity=2,
    add_ons={"샷": 2},
    customer_requests=["더 진하게 해주세요"]
)
add_response = order_manager.add_item(item1)
print(json.dumps(add_response, ensure_ascii=False))

# Update an item in the order
item_updated = OrderItem(
    drink="아메리카노",
    size="미디움",
    temperature="아이스",
    quantity=1,
    add_ons={"샷": 1},
    customer_requests=["시럽 추가해주세요"]
)
update_response = order_manager.update_item(0, item_updated)
print(json.dumps(update_response, ensure_ascii=False))

# Get a summary of the order
summary_response = order_manager.get_order_summary()
print(json.dumps(summary_response, ensure_ascii=False))

# Cancel the order
cancel_response = order_manager.cancel_order()
print(json.dumps(cancel_response, ensure_ascii=False))


{"status": "success", "message": "Item added successfully."}
{"status": "success", "message": "Item updated successfully."}
{"status": "success", "order_summary": {"order_id": 1, "order_items": [{"drink": "아메리카노", "size": "미디움", "temperature": "아이스", "quantity": 1, "add_ons": {"샷": 1}, "customer_requests": ["시럽 추가해주세요"]}]}}
{"status": "success", "message": "Order cancelled successfully."}


In [ ]:
import json
from typing import List, Dict

# Assuming we have the Order, OrderItem, and OrderManager classes from the earlier provided code
# We also add save and load functionality to OrderManager

class Order:
    def __init__(self, order_id: int):
        self.order_id = order_id
        self.order_items = []

    def to_dict(self):
        return {
            "order_id": self.order_id,
            "order_items": [item.to_dict() for item in self.order_items]
        }

class OrderItem:
    def __init__(self, drink: str, size: str, temperature: str, quantity: int, add_ons: Dict[str, int], customer_requests: List[str]):
        self.drink = drink
        self.size = size
        self.temperature = temperature
        self.quantity = quantity
        self.add_ons = add_ons
        self.customer_requests = customer_requests

    def to_dict(self):
        return {
            "drink": self.drink,
            "size": self.size,
            "temperature": self.temperature,
            "quantity": self.quantity,
            "add_ons": self.add_ons,
            "customer_requests": self.customer_requests
        }

class OrderManager:
    def __init__(self):
        self.orders = {}
        self.current_order_id = 0
        self.load_orders_from_file()

    def create_new_order(self):
        self.current_order_id += 1
        new_order = Order(self.current_order_id)
        self.orders[self.current_order_id] = new_order
        self.save_orders_to_file()
        return new_order

    def add_item(self, order_id: int, item: OrderItem):
        if order_id in self.orders:
            self.orders[order_id].order_items.append(item)
            self.save_orders_to_file()
            return {"status": "success", "message": "Item added successfully."}
        else:
            return {"status": "error", "message": "Order not found."}

    def update_item_partial(self, order_id: int, item_index: int, changes: Dict):
        if order_id in self.orders:
            if 0 <= item_index < len(self.orders[order_id].order_items):
                item = self.orders[order_id].order_items[item_index]

                # Apply the changes to the existing item
                if "drink" in changes:
                    item.drink = changes["drink"]
                if "size" in changes:
                    item.size = changes["size"]
                if "temperature" in changes:
                    item.temperature = changes["temperature"]
                if "quantity" in changes:
                    item.quantity = changes["quantity"]
                if "add_ons" in changes:
                    item.add_ons.update(changes["add_ons"])
                if "customer_requests" in changes:
                    item.customer_requests.extend(changes["customer_requests"])

                self.save_orders_to_file()
                return {"status": "success", "message": "Item updated successfully."}
            else:
                return {"status": "error", "message": "Item index out of range."}
        else:
            return {"status": "error", "message": "Order not found."}

    def save_orders_to_file(self, file_path="orders.json"):
        with open(file_path, "w", encoding="utf-8") as f:
            json.dump({order_id: order.to_dict() for order_id, order in self.orders.items()}, f, ensure_ascii=False, indent=4)

    def load_orders_from_file(self, file_path="orders.json"):
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                orders_data = json.load(f)
                for order_id, order_data in orders_data.items():
                    order = Order(order_id=int(order_id))
                    order.order_items = [
                        OrderItem(
                            drink=item["drink"],
                            size=item["size"],
                            temperature=item["temperature"],
                            quantity=item["quantity"],
                            add_ons=item["add_ons"],
                            customer_requests=item["customer_requests"]
                        )
                        for item in order_data["order_items"]
                    ]
                    self.orders[int(order_id)] = order
        except FileNotFoundError:
            # File doesn't exist yet
            self.orders = {}

# Initialize OrderManager
order_manager = OrderManager()
